In [1]:
import math
import pandas as pd
import os

from pyarrow import csv
import pyarrow as pa
import pyarrow.parquet as pq
import time

### csv, parquet

- [parquet 공식문서](https://arrow.apache.org/docs/python/parquet.html)
- [Python parquet 타입으로 읽기, 쓰기](https://owening2.tistory.com/19)
- [parquet? / csv, parquet 파일 불러오기 테스트](https://velog.io/@freetix/parquet-csv-parquet-%ED%8C%8C%EC%9D%BC-%EB%B6%88%EB%9F%AC%EC%98%A4%EA%B8%B0-%ED%85%8C%EC%8A%A4%ED%8A%B8
)


In [4]:
os.getcwd()

'/home/shjj08choi4/finance_mlops/jupyter'

In [6]:
kor_stock_ohlcv = pd.read_csv('../data_crawler/kor_stock_ohlcv/kor_stock_ohlcv.csv')

/tmp/ipykernel_1143244/1869254842.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  kor_stock_ohlcv = pd.read_csv('../data_crawler/kor_stock_ohlcv/kor_stock_ohlcv.csv')


In [12]:
kor_stock_ohlcv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3366395 entries, 0 to 3366394
Data columns (total 8 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   date                     object 
 1   open                     int64  
 2   high                     int64  
 3   low                      int64  
 4   close                    int64  
 5   volume                   int64  
 6   price_change_percentage  float64
 7   ticker                   object 
dtypes: float64(1), int64(5), object(2)
memory usage: 205.5+ MB


In [13]:
kor_stock_ohlcv.head()

,date,open,high,low,close,volume,price_change_percentage,ticker
0,2018-01-02,6980,6990,6850,6970,60294,0.000000,095570
1,2018-01-03,6970,7000,6910,6920,45848,-0.717360,095570
2,2018-01-04,6990,7100,6920,7070,53176,2.167630,095570
3,2018-01-05,7080,7300,7040,7250,58573,2.545969,095570
4,2018-01-08,7330,7330,7100,7130,47104,-1.655172,095570


In [14]:
kor_stock_ohlcv['ticker'] = kor_stock_ohlcv['ticker'].astype('str')

#### parquet로 저장하기

In [16]:

table_from_pandas = pa.Table.from_pandas(kor_stock_ohlcv,preserve_index = False)
pq.write_table(table_from_pandas, '../data_crawler/kor_stock_ohlcv/kor_stock_ohlcv.parquet')

#### parquet 읽어오기

In [17]:
t = pq.read_table('../data_crawler/kor_stock_ohlcv/kor_stock_ohlcv.parquet')

In [18]:
t.to_pandas()

,date,open,high,low,close,volume,price_change_percentage,ticker
0,2018-01-02,6980,6990,6850,6970,60294,0.000000,095570
1,2018-01-03,6970,7000,6910,6920,45848,-0.717360,095570
2,2018-01-04,6990,7100,6920,7070,53176,2.167630,095570
3,2018-01-05,7080,7300,7040,7250,58573,2.545969,095570
4,2018-01-08,7330,7330,7100,7130,47104,-1.655172,095570
...,...,...,...,...,...,...,...,...
3366390,2023-11-30,6220,6330,6210,6330,12703,2.261712,238490
3366391,2023-12-01,6330,6490,6290,6410,31437,1.263823,238490
3366392,2023-12-04,6450,6500,6380,6430,19005,0.312012,238490
3366393,2023-12-05,6480,6480,6250,6360,21136,-1.088647,238490
